In [1]:
import pymongo as mongo
import pandas as pd
from pandas_profiling import ProfileReport

In [2]:
# connect to database

client = mongo.MongoClient("mongodb://localhost:27017")


# Select the database and collection
db = client["crm"]

crm_activity = db["crm_activity"]
crm_user = db["crm_user"]
crm_pipeline = db["crm_pipeline"]
crm_organization = db["crm_organization"]
crm_deal=db["crm_deal"]
crm_customer = db["crm_customer"]
crm_activity_type = db["crm_activity_type"]

# create a dataframe from the collection
df_activity = pd.DataFrame(list(crm_activity.find()))
df_user = pd.DataFrame(list(crm_user.find()))
df_pipeline = pd.DataFrame(list(crm_pipeline.find()))
df_organization = pd.DataFrame(list(crm_organization.find()))
df_deal = pd.DataFrame(list(crm_deal.find()))
df_customer = pd.DataFrame(list(crm_customer.find()))
df_activity_type = pd.DataFrame(list(crm_activity_type.find()))

df_user.head(5)

,_id,id,crmId,crmIsAdmin,crmName,crmDefaultCurrency,crmTimezoneOffset,crmEmail,crmPhone,crmLastLogin,crmCreateDate,createDate,crmIsActive,isDeleted,companyId
0,64902f167cc8b1037a62bdd7,1,3204921,0,Amanda Partosch,EUR,+02:00,amanda@casavi.de,None,2017-11-27 09:24:27,None,2022-01-27 14:13:34,1,0,5
1,64902f167cc8b1037a62bdd8,2,9033176,0,Anja Stolz,EUR,+02:00,anja.stolz@casavi.de,None,2019-09-09 12:04:28,None,2022-01-27 14:13:34,1,0,5
2,64902f167cc8b1037a62bdd9,3,11406786,0,Anna Gassmann,EUR,+02:00,anna.gassmann@casavi.de,None,2020-11-25 15:16:17,None,2022-01-27 14:13:34,1,0,5
3,64902f167cc8b1037a62bdda,4,12654440,0,Arne Leppelsack,EUR,+02:00,arne.leppelsack@casavi.de,None,2021-10-14 06:47:40,None,2022-01-27 14:13:34,1,0,5
4,64902f167cc8b1037a62bddb,5,11353976,0,Benjamin Grotepass,EUR,+02:00,benjamin.grotepass@casavi.de,None,2022-05-13 06:42:47,None,2022-01-27 14:13:34,1,0,5


In [3]:
#Merge the datasets
# Merge the datasets
#df = pd.merge(df_activity, df_user, on='crmUserId', suffixes=('_activity', '_user'))
df = pd.merge(df_activity, df_user, left_on='crmUserId', right_on='crmId', suffixes=('_activity', '_user'))
df = pd.merge(df, df_pipeline, left_on='crmId_activity', right_on='crmId', suffixes=('', '_pipeline'))
df = pd.merge(df, df_organization, left_on='crmId_activity', right_on='crmId', suffixes=('', '_organization'))
df = pd.merge(df, df_deal, left_on='crmDealId', right_on='crmId', suffixes=('', '_deal'))
df = pd.merge(df, df_customer, left_on='crmCustomerId', right_on='crmId', suffixes=('', '_customer'))
df = pd.merge(df, df_activity_type, left_on='crmType', right_on='key_string', suffixes=('', '_activity_type'))

df.head(5)  # This will display the first few rows of the merged DataFrame

,_id_activity,id_activity,crmId_activity,crmUserId,crmDealId,crmLeadId,crmType,crmAddTime,crmUpdateTime,crmDueDate,...,crmZipCode_customer,crmPhoneNumber,_id_activity_type,id_activity_type,key_string,name,staticActivityTypeId,companyId_activity_type,createDate_activity_type,isDeleted_activity_type
0,64902df27cc8b1037a57f1e6,10472,11,686380.0,9.0,None,deadline,2015-03-04 09:01:14,2015-10-12 10:16:31,2015-10-12,...,NaN,+49 7352 93205,64902ec17cc8b1037a5ec862,4,deadline,Deadline,4.0,11,2022-05-03 09:36:42,0
1,64902df27cc8b1037a57f1e6,10472,11,686380.0,9.0,None,deadline,2015-03-04 09:01:14,2015-10-12 10:16:31,2015-10-12,...,NaN,+49 7352 93205,64902ec17cc8b1037a5ec86b,13,deadline,Frist,NaN,9,2022-05-03 09:37:29,0
2,64902df27cc8b1037a57f1e6,10472,11,686380.0,9.0,None,deadline,2015-03-04 09:01:14,2015-10-12 10:16:31,2015-10-12,...,NaN,+49 7352 93205,64902ec17cc8b1037a5ec87f,33,deadline,Frist,3.0,12,2022-05-04 18:40:43,0
3,64902df27cc8b1037a57f1e6,10472,11,686380.0,9.0,None,deadline,2015-03-04 09:01:14,2015-10-12 10:16:31,2015-10-12,...,NaN,+49 7352 93205,64902ec17cc8b1037a5ec888,42,deadline,Deadline,1.0,13,2022-05-05 13:40:38,0
4,64902df27cc8b1037a57f1e6,10472,11,686380.0,9.0,None,deadline,2015-03-04 09:01:14,2015-10-12 10:16:31,2015-10-12,...,NaN,+49 7352 93205,64902ec17cc8b1037a5ec88e,48,deadline,Deadline,NaN,14,2022-05-06 04:06:06,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15740175 entries, 0 to 15740174
Data columns (total 96 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   _id_activity               object        
 1   id_activity                int64         
 2   crmId_activity             int64         
 3   crmUserId                  float64       
 4   crmDealId                  float64       
 5   crmLeadId                  object        
 6   crmType                    object        
 7   crmAddTime                 object        
 8   crmUpdateTime              object        
 9   crmDueDate                 datetime64[ns]
 10  crmSubject                 object        
 11  crmDone                    object        
 12  isDeleted_activity         object        
 13  createDate_activity        object        
 14  companyId_activity         object        
 15  crmDueTime                 object        
 16  crmDuration                object 

In [5]:
df.describe()

,id_activity,crmId_activity,crmUserId,crmDealId,id_user,crmId_user,crmIsAdmin,crmIsActive,isDeleted_user,companyId_user,...,crmCity_customer,crmStreet_customer,isDeleted_customer,companyId_customer,crmOrgId_customer,crmZipCode_customer,id_activity_type,staticActivityTypeId,companyId_activity_type,isDeleted_activity_type
count,1.574018e+07,1.574018e+07,1.574018e+07,1.574018e+07,1.574018e+07,1.574018e+07,15740175.0,15740175.0,15740175.0,1.574018e+07,...,0.0,0.0,1.574018e+07,1.574018e+07,1.359994e+07,0.0,1.574018e+07,1.268138e+07,1.574018e+07,15740175.0
mean,8.058170e+05,8.242705e+00,1.123143e+07,1.149271e+01,3.921987e+06,1.123143e+07,0.0,1.0,0.0,8.504028e+00,...,NaN,NaN,4.757253e-04,1.106471e+01,8.326435e+02,NaN,6.282177e+01,1.296633e+00,1.700008e+01,0.0
std,5.840951e+06,4.793942e+00,3.326152e+06,3.990577e+01,6.388749e+06,3.326152e+06,0.0,0.0,0.0,6.733963e+00,...,NaN,NaN,2.180594e-02,8.265200e+00,3.832680e+03,NaN,3.474287e+01,6.971805e-01,1.034075e+01,0.0
min,1.003600e+04,1.000000e+00,5.196600e+05,1.000000e+00,3.900000e+01,5.196600e+05,0.0,1.0,0.0,2.000000e+00,...,NaN,NaN,0.000000e+00,2.000000e+00,1.000000e+00,NaN,1.000000e+00,1.000000e+00,4.000000e+00,0.0
25%,1.986120e+05,5.000000e+00,1.218294e+07,5.000000e+00,1.130000e+02,1.218294e+07,0.0,1.0,0.0,3.000000e+00,...,NaN,NaN,0.000000e+00,5.000000e+00,5.000000e+00,NaN,3.900000e+01,1.000000e+00,1.100000e+01,0.0
50%,2.285530e+05,7.000000e+00,1.219530e+07,7.000000e+00,1.230000e+02,1.219530e+07,0.0,1.0,0.0,5.000000e+00,...,NaN,NaN,0.000000e+00,9.000000e+00,8.000000e+00,NaN,6.900000e+01,1.000000e+00,1.400000e+01,0.0
75%,3.005370e+05,9.000000e+00,1.251282e+07,1.100000e+01,1.429778e+07,1.251282e+07,0.0,1.0,0.0,1.200000e+01,...,NaN,NaN,0.000000e+00,1.400000e+01,6.200000e+01,NaN,8.800000e+01,1.000000e+00,1.900000e+01,0.0
max,6.090265e+07,2.000000e+01,1.349794e+07,3.171000e+03,1.516801e+07,1.349794e+07,0.0,1.0,0.0,4.200000e+01,...,NaN,NaN,1.000000e+00,4.200000e+01,4.215400e+04,NaN,1.150000e+02,4.000000e+00,4.200000e+01,0.0


In [6]:
df.shape

(15740175, 96)

In [7]:
#df.isnull().sum() 
# Remove columns with no data
df = df.drop(['crmCity_customer', 'crmStreet_customer', 'crmZipCode_customer'], axis=1)
df.describe()


,id_activity,crmId_activity,crmUserId,crmDealId,id_user,crmId_user,crmIsAdmin,crmIsActive,isDeleted_user,companyId_user,...,crmPipelineId,id_customer,crmId_customer,isDeleted_customer,companyId_customer,crmOrgId_customer,id_activity_type,staticActivityTypeId,companyId_activity_type,isDeleted_activity_type
count,1.574018e+07,1.574018e+07,1.574018e+07,1.574018e+07,1.574018e+07,1.574018e+07,15740175.0,15740175.0,15740175.0,1.574018e+07,...,1.574018e+07,1.574018e+07,1.574018e+07,1.574018e+07,1.574018e+07,1.359994e+07,1.574018e+07,1.268138e+07,1.574018e+07,15740175.0
mean,8.058170e+05,8.242705e+00,1.123143e+07,1.149271e+01,3.921987e+06,1.123143e+07,0.0,1.0,0.0,8.504028e+00,...,1.828848e+00,8.351617e+04,2.167157e+02,4.757253e-04,1.106471e+01,8.326435e+02,6.282177e+01,1.296633e+00,1.700008e+01,0.0
std,5.840951e+06,4.793942e+00,3.326152e+06,3.990577e+01,6.388749e+06,3.326152e+06,0.0,0.0,0.0,6.733963e+00,...,2.379738e+00,3.506778e+04,1.845754e+03,2.180594e-02,8.265200e+00,3.832680e+03,3.474287e+01,6.971805e-01,1.034075e+01,0.0
min,1.003600e+04,1.000000e+00,5.196600e+05,1.000000e+00,3.900000e+01,5.196600e+05,0.0,1.0,0.0,2.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,0.0
25%,1.986120e+05,5.000000e+00,1.218294e+07,5.000000e+00,1.130000e+02,1.218294e+07,0.0,1.0,0.0,3.000000e+00,...,1.000000e+00,7.215400e+04,5.000000e+00,0.000000e+00,5.000000e+00,5.000000e+00,3.900000e+01,1.000000e+00,1.100000e+01,0.0
50%,2.285530e+05,7.000000e+00,1.219530e+07,7.000000e+00,1.230000e+02,1.219530e+07,0.0,1.0,0.0,5.000000e+00,...,1.000000e+00,7.294100e+04,8.000000e+00,0.000000e+00,9.000000e+00,8.000000e+00,6.900000e+01,1.000000e+00,1.400000e+01,0.0
75%,3.005370e+05,9.000000e+00,1.251282e+07,1.100000e+01,1.429778e+07,1.251282e+07,0.0,1.0,0.0,1.200000e+01,...,1.000000e+00,1.102110e+05,1.800000e+01,0.000000e+00,1.400000e+01,6.200000e+01,8.800000e+01,1.000000e+00,1.900000e+01,0.0
max,6.090265e+07,2.000000e+01,1.349794e+07,3.171000e+03,1.516801e+07,1.349794e+07,0.0,1.0,0.0,4.200000e+01,...,1.800000e+01,1.371060e+05,4.355800e+04,1.000000e+00,4.200000e+01,4.215400e+04,1.150000e+02,4.000000e+00,4.200000e+01,0.0


In [8]:
#missing_crmOrgId_customer = df[df['crmOrgId_customer'].isnull()]
#print(missing_crmOrgId_customer)
df = df.dropna(subset=['crmOrgId_customer'])
df.describe()

,id_activity,crmId_activity,crmUserId,crmDealId,id_user,crmId_user,crmIsAdmin,crmIsActive,isDeleted_user,companyId_user,...,crmPipelineId,id_customer,crmId_customer,isDeleted_customer,companyId_customer,crmOrgId_customer,id_activity_type,staticActivityTypeId,companyId_activity_type,isDeleted_activity_type
count,1.359994e+07,1.359994e+07,1.359994e+07,1.359994e+07,1.359994e+07,1.359994e+07,13599943.0,13599943.0,13599943.0,1.359994e+07,...,1.359994e+07,1.359994e+07,1.359994e+07,13599943.0,1.359994e+07,1.359994e+07,1.359994e+07,1.094837e+07,1.359994e+07,13599943.0
mean,8.617333e+05,8.396803e+00,1.123112e+07,1.163652e+01,3.859349e+06,1.123112e+07,0.0,1.0,0.0,8.490128e+00,...,1.850145e+00,8.514568e+04,2.295880e+02,0.0,1.140669e+01,8.326435e+02,6.278740e+01,1.283075e+00,1.700009e+01,0.0
std,6.122462e+06,4.787947e+00,3.347939e+06,3.892205e+01,6.357692e+06,3.347939e+06,0.0,0.0,0.0,6.805570e+00,...,2.417662e+00,3.654479e+04,1.956990e+03,0.0,8.731901e+00,3.832680e+03,3.475016e+01,6.881598e-01,1.034080e+01,0.0
min,1.003600e+04,1.000000e+00,5.196600e+05,1.000000e+00,3.900000e+01,5.196600e+05,0.0,1.0,0.0,2.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,0.0,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,0.0
25%,1.986080e+05,5.000000e+00,1.218294e+07,6.000000e+00,1.130000e+02,1.218294e+07,0.0,1.0,0.0,3.000000e+00,...,1.000000e+00,7.215800e+04,6.000000e+00,0.0,4.000000e+00,5.000000e+00,3.900000e+01,1.000000e+00,1.100000e+01,0.0
50%,2.285510e+05,7.000000e+00,1.219530e+07,7.000000e+00,1.230000e+02,1.219530e+07,0.0,1.0,0.0,5.000000e+00,...,1.000000e+00,7.294100e+04,8.000000e+00,0.0,1.100000e+01,8.000000e+00,6.900000e+01,1.000000e+00,1.400000e+01,0.0
75%,3.002000e+05,9.000000e+00,1.251282e+07,1.100000e+01,1.429778e+07,1.251282e+07,0.0,1.0,0.0,1.200000e+01,...,1.000000e+00,1.313920e+05,1.800000e+01,0.0,1.600000e+01,6.200000e+01,8.800000e+01,1.000000e+00,1.900000e+01,0.0
max,6.090265e+07,2.000000e+01,1.349794e+07,3.171000e+03,1.516801e+07,1.349794e+07,0.0,1.0,0.0,4.200000e+01,...,1.800000e+01,1.367030e+05,4.355800e+04,0.0,4.200000e+01,4.215400e+04,1.150000e+02,4.000000e+00,4.200000e+01,0.0


# Feature Selection
### It seems like columns such as "crmStatus", "crmValue", "crmAddTime_deal", "crmUpdateTime_deal", "crmValueCurrency", "crmLostTime", "crmWonTime" could be important.

In [9]:
df_fs = df[['crmStatus', 'crmValue', 'crmAddTime_deal', 'crmUpdateTime_deal', 'crmValueCurrency', 'crmLostTime', 'crmWonTime']]

df_fs.head()


,crmStatus,crmValue,crmAddTime_deal,crmUpdateTime_deal,crmValueCurrency,crmLostTime,crmWonTime
0,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,None,2015-10-30 16:41:34
1,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,None,2015-10-30 16:41:34
2,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,None,2015-10-30 16:41:34
3,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,None,2015-10-30 16:41:34
4,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,None,2015-10-30 16:41:34


In [10]:
unique_values = df_fs['crmStatus'].unique()
print(unique_values)

['won' 'lost' 'open']


In [11]:
#df_fs['isWon'] = df_fs['crmStatus'].apply(lambda x: 1 if x == 'won' else 0)
df_fs.loc[:, 'isWon'] = df_fs['crmStatus'].apply(lambda x: 1 if x == 'won' else 0)

df_fs.head()

C:\Users\nakhu\AppData\Local\Temp\ipykernel_4752\2854412191.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fs.loc[:, 'isWon'] = df_fs['crmStatus'].apply(lambda x: 1 if x == 'won' else 0)


,crmStatus,crmValue,crmAddTime_deal,crmUpdateTime_deal,crmValueCurrency,crmLostTime,crmWonTime,isWon
0,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,None,2015-10-30 16:41:34,1
1,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,None,2015-10-30 16:41:34,1
2,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,None,2015-10-30 16:41:34,1
3,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,None,2015-10-30 16:41:34,1
4,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,None,2015-10-30 16:41:34,1


In [12]:
df_fs[['crmAddTime_deal', 'crmUpdateTime_deal', 'crmWonTime']].isnull().sum()


crmAddTime_deal              0
crmUpdateTime_deal           0
crmWonTime            11163539
dtype: int64

# Time conversions

In [16]:
# SettingWithCopyWarning
# Time conversion
df_fs = df_fs.copy()  # create a copy of the DataFrame to avoid modifying the original df
df_fs['crmAddTime_deal'] = pd.to_datetime(df_fs['crmAddTime_deal'])
df_fs['crmWonTime'] = pd.to_datetime(df_fs['crmWonTime'])
df_fs['crmUpdateTime_deal'] = pd.to_datetime(df_fs['crmUpdateTime_deal'])
df_fs['crmLostTime'] = pd.to_datetime(df_fs['crmLostTime'])

# Convert the columns to datetime
df_fs['crmAddTime_deal'] = pd.to_datetime(df_fs['crmAddTime_deal'])
df_fs['crmWonTime'] = pd.to_datetime(df_fs['crmWonTime'])
df_fs['crmUpdateTime_deal'] = pd.to_datetime(df_fs['crmUpdateTime_deal'])
df_fs['crmLostTime'] = pd.to_datetime(df_fs['crmLostTime'])

# Extract datetime elements from crmAddTime_deal
df_fs['crmAddTime_deal_year'] = df_fs['crmAddTime_deal'].dt.year
df_fs['crmAddTime_deal_month'] = df_fs['crmAddTime_deal'].dt.month
df_fs['crmAddTime_deal_day'] = df_fs['crmAddTime_deal'].dt.day
df_fs['crmAddTime_deal_hour'] = df_fs['crmAddTime_deal'].dt.hour
df_fs['crmAddTime_deal_minute'] = df_fs['crmAddTime_deal'].dt.minute
df_fs['crmAddTime_deal_second'] = df_fs['crmAddTime_deal'].dt.second
df_fs['crmAddTime_deal_day_of_week'] = df_fs['crmAddTime_deal'].dt.dayofweek

# Extract datetime elements from crmWonTime
df_fs['crmWonTime_year'] = df_fs['crmWonTime'].dt.year
df_fs['crmWonTime_month'] = df_fs['crmWonTime'].dt.month
df_fs['crmWonTime_day'] = df_fs['crmWonTime'].dt.day
df_fs['crmWonTime_hour'] = df_fs['crmWonTime'].dt.hour
df_fs['crmWonTime_minute'] = df_fs['crmWonTime'].dt.minute
df_fs['crmWonTime_second'] = df_fs['crmWonTime'].dt.second
df_fs['crmWonTime_day_of_week'] = df_fs['crmWonTime'].dt.dayofweek

# Extract datetime elements from crmUpdateTime_deal
df_fs['crmUpdateTime_deal_year'] = df_fs['crmUpdateTime_deal'].dt.year
df_fs['crmUpdateTime_deal_month'] = df_fs['crmUpdateTime_deal'].dt.month
df_fs['crmUpdateTime_deal_day'] = df_fs['crmUpdateTime_deal'].dt.day
df_fs['crmUpdateTime_deal_hour'] = df_fs['crmUpdateTime_deal'].dt.hour
df_fs['crmUpdateTime_deal_minute'] = df_fs['crmUpdateTime_deal'].dt.minute
df_fs['crmUpdateTime_deal_second'] = df_fs['crmUpdateTime_deal'].dt.second
df_fs['crmUpdateTime_deal_day_of_week'] = df_fs['crmUpdateTime_deal'].dt.dayofweek

# Extract datetime elements from crmLostTime
df_fs['crmLostTime_year'] = df_fs['crmLostTime'].dt.year
df_fs['crmLostTime_year'] = df_fs['crmLostTime_year'].fillna(df_fs['crmLostTime_year'].mean())

df_fs['crmLostTime_month'] = df_fs['crmLostTime'].dt.month
df_fs['crmLostTime_month'] = df_fs['crmLostTime_month'].fillna(df_fs['crmLostTime_month'].mean())

df_fs['crmLostTime_day'] = df_fs['crmLostTime'].dt.day
df_fs['crmLostTime_day'] = df_fs['crmLostTime_day'].fillna(df_fs['crmLostTime_day'].mean())

df_fs['crmLostTime_hour'] = df_fs['crmLostTime'].dt.hour
df_fs['crmLostTime_hour'] = df_fs['crmLostTime_hour'].fillna(df_fs['crmLostTime_hour'].mean())

df_fs['crmLostTime_minute'] = df_fs['crmLostTime'].dt.minute
df_fs['crmLostTime_minute'] = df_fs['crmLostTime_minute'].fillna(df_fs['crmLostTime_minute'].mean())

df_fs['crmLostTime_second'] = df_fs['crmLostTime'].dt.second
df_fs['crmLostTime_second'] = df_fs['crmLostTime_second'].fillna(df_fs['crmLostTime_second'].mean())

df_fs['crmLostTime_day_of_week'] = df_fs['crmLostTime'].dt.dayofweek
df_fs['crmLostTime_day_of_week'] = df_fs['crmLostTime_day_of_week'].fillna(df_fs['crmLostTime_day_of_week'].mean())

In [17]:
df_fs.head()

,crmStatus,crmValue,crmAddTime_deal,crmUpdateTime_deal,crmValueCurrency,crmLostTime,crmWonTime,isWon,crmAddTime_deal_year,crmAddTime_deal_month,...,crmUpdateTime_deal_minute,crmUpdateTime_deal_second,crmUpdateTime_deal_day_of_week,crmLostTime_year,crmLostTime_month,crmLostTime_day,crmLostTime_hour,crmLostTime_minute,crmLostTime_second,crmLostTime_day_of_week
0,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,NaT,2015-10-30 16:41:34,1,2015,3,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
1,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,NaT,2015-10-30 16:41:34,1,2015,3,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
2,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,NaT,2015-10-30 16:41:34,1,2015,3,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
3,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,NaT,2015-10-30 16:41:34,1,2015,3,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
4,won,120,2015-03-04 09:00:21,2021-05-20 16:53:59,EUR,NaT,2015-10-30 16:41:34,1,2015,3,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363


In [18]:
df_fs = df_fs.drop(['crmAddTime_deal', 'crmUpdateTime_deal','crmLostTime', 'crmWonTime', 'crmStatus'], axis=1)
df_fs.head()


,crmValue,crmValueCurrency,isWon,crmAddTime_deal_year,crmAddTime_deal_month,crmAddTime_deal_day,crmAddTime_deal_hour,crmAddTime_deal_minute,crmAddTime_deal_second,crmAddTime_deal_day_of_week,...,crmUpdateTime_deal_minute,crmUpdateTime_deal_second,crmUpdateTime_deal_day_of_week,crmLostTime_year,crmLostTime_month,crmLostTime_day,crmLostTime_hour,crmLostTime_minute,crmLostTime_second,crmLostTime_day_of_week
0,120,EUR,1,2015,3,4,9,0,21,2,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
1,120,EUR,1,2015,3,4,9,0,21,2,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
2,120,EUR,1,2015,3,4,9,0,21,2,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
3,120,EUR,1,2015,3,4,9,0,21,2,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
4,120,EUR,1,2015,3,4,9,0,21,2,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363


In [19]:
df_fs = df_fs.drop(['crmValueCurrency'], axis=1)
df_fs.head()

,crmValue,isWon,crmAddTime_deal_year,crmAddTime_deal_month,crmAddTime_deal_day,crmAddTime_deal_hour,crmAddTime_deal_minute,crmAddTime_deal_second,crmAddTime_deal_day_of_week,crmWonTime_year,...,crmUpdateTime_deal_minute,crmUpdateTime_deal_second,crmUpdateTime_deal_day_of_week,crmLostTime_year,crmLostTime_month,crmLostTime_day,crmLostTime_hour,crmLostTime_minute,crmLostTime_second,crmLostTime_day_of_week
0,120,1,2015,3,4,9,0,21,2,2015.0,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
1,120,1,2015,3,4,9,0,21,2,2015.0,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
2,120,1,2015,3,4,9,0,21,2,2015.0,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
3,120,1,2015,3,4,9,0,21,2,2015.0,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363
4,120,1,2015,3,4,9,0,21,2,2015.0,...,53,59,3,2020.086483,6.161892,17.25204,11.971806,27.063901,22.862169,2.387363


In [20]:
pd.set_option('display.max_columns', None)
print(df_fs)


          crmValue  isWon  crmAddTime_deal_year  crmAddTime_deal_month  \
0              120      1                  2015                      3   
1              120      1                  2015                      3   
2              120      1                  2015                      3   
3              120      1                  2015                      3   
4              120      1                  2015                      3   
...            ...    ...                   ...                    ...   
15740170     36000      0                  2021                      8   
15740171     36000      0                  2021                      8   
15740172     36000      0                  2021                      8   
15740173     36000      0                  2021                      8   
15740174     36000      0                  2021                      8   

          crmAddTime_deal_day  crmAddTime_deal_hour  crmAddTime_deal_minute  \
0                           4   

### Split your data into a training set and a test set

In [21]:
from sklearn.model_selection import train_test_split

# Assuming df_fs is your DataFrame and 'isWon' is the target variable
X = df_fs.drop('isWon', axis=1)
y = df_fs['isWon']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Use random forest and xgboost to see the best

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Create an imputer with the strategy set to 'mean'
imputer = SimpleImputer(strategy='mean')

# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)


# Now fit your model - Train the classifier
imputer.fit(X_train)
X_train = imputer.transform(X_train)
rf.fit(X_train, y_train)

# Evaluate the model
rf_score = rf.score(X_test, y_test)

print(f"Random Forest accuracy: {rf_score*100:.2f}%")

c:\Users\nakhu\anaconda3\envs\tf_env\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values